In [1]:
# header files needed
import torch
import torch.nn as nn
import torchvision
from dataset import *
from transforms import *
import numpy as np
import glob
#import cv2

In [2]:
# ensure the experiment produces same result on each run
np.random.seed(1234)
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
transforms = torchvision.transforms.Compose([ToTensor(), Normalize()])
train_dataset = SegDataset("/content/drive/My Drive/CrackForest/", "Images", "Masks", transform=transforms, seed=100, fraction=0.2, subset="Train")
val_dataset = SegDataset("/content/drive/My Drive/CrackForest/", "Images", "Masks", transform=transforms, seed=100, fraction=0.2, subset="Val")

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=True, num_workers=8)

In [ ]:
# loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.MSELoss()

In [ ]:
# hyper-parameters and hard-coded values
num_epochs = 100
lr = 1e-4
wd = 5e-6

In [ ]:
model = torchvision.models.segmentation.fcn_resnet101(pretrained=True)
model.classifier = torchvision.models.segmentation.fcn.FCNHead(2048, 1)
model.to(device)
print(model)

In [ ]:
# create optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=0.95)

In [ ]:
train_loss_list = []
train_accuracy_list = []
val_loss_list = []
val_accuracy_list = []
best_metric = -1
best_metric_epoch = -1


# training and val loop
for epoch in range(0, num_epochs):

  # train
  model.train()
  train_loss = 0.0
  train_accuracy = 0.0
  correct = 0.0
  total = 0.0
  for _, sample in enumerate(train_loader):
    
    # if cuda
    images = sample["image"].to(device)
    labels = sample["mask"].to(device)
    
    # get loss
    optimizer.zero_grad()
    outputs = model(images)['out']
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

    # convert outputs and labels to rank-1 tensor
    outputs = outputs.flatten().cpu()
    labels = labels.flatten().cpu()

    # update training_accuracy
    total += ((labels == labels) * (labels > 0)).sum()
    correct += ((labels > 0) * (outputs > 0.1)).sum()

  # update training_loss, training_accuracy and training_iou 
  train_loss = train_loss / float(len(train_loader))
  train_accuracy = float(correct) / float(total)
  train_loss_list.append(train_loss)
  train_accuracy_list.append(train_accuracy)

  # evaluation code
  model.eval()
  val_loss = 0.0
  val_accuracy = 0.0
  val_iou = 0.0
  correct = 0.0
  total = 0.0
  for _, sample in enumerate(val_loader):
    with torch.no_grad():

      # if cuda
      images = sample["image"].to(device)
      labels = sample["mask"].to(device)

      # get loss
      outputs = model(images)['out']
      loss = criterion(outputs, labels)
      val_loss += loss.item()

      # convert outputs and labels to rank-1 tensor
      outputs = outputs.flatten().cpu()
      labels = labels.flatten().cpu()

      # update val_accuracy
      total += ((labels == labels) * (labels > 0)).sum()
      correct += ((labels > 0) * (outputs > 0.1)).sum()

  # update val_loss, val_accuracy and val_iou 
  val_loss = val_loss / float(len(val_loader))
  val_accuracy = float(correct) / float(total)
  val_loss_list.append(val_loss)
  val_accuracy_list.append(val_accuracy)

  
  # early stopping
  if(best_metric < float(val_accuracy) and epoch >= 10):
    best_metric = float(val_accuracy)
    best_metric_epoch = epoch
    torch.save(model.state_dict(), "/content/drive/My Drive/best_model.pth")

  print()
  print("Epoch: " + str(epoch))
  print("Training Loss: " + str(train_loss) + "    Validation Loss: " + str(val_loss))
  print("Training Accuracy: " + str(train_accuracy) + "    Validation Accuracy: " + str(val_accuracy))
  print()

In [3]:
import matplotlib.pyplot as plt

In [4]:
e = []
for index in range(0, num_epochs):
  e.append(index)

In [ ]:
plt.plot(e, train_loss_list)

In [ ]:
plt.plot(e, val_loss_list)